<a href="https://colab.research.google.com/github/kotosham/sleep-stages/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/3.csv')
data3 = data3.drop(columns=['Time,s'], axis=1)
df3 = pd.DataFrame(data3)

In [4]:
dataframes = []

for i in range(1, 8):
  if i == 3:
    continue
  url = f'https://raw.githubusercontent.com/kotosham/sleep-stages/refs/heads/main/{i}.csv'
  data = pd.read_csv(url)
  data = data.drop(columns=['Time,s'], axis=1)
  dataframes.append(pd.DataFrame(data))

dataframes.insert(2, df3)
df1 = dataframes[0]
df2 = dataframes[1]
df4 = dataframes[3]
df5 = dataframes[4]
df6 = dataframes[5]
df7 = dataframes[6]

<ipython-input-4-6d7dece21c35>:7: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)
<ipython-input-4-6d7dece21c35>:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


In [5]:
df4['1 Anadh'] = df4['1 Anadh'].replace('#DIV/0!', '0')
df4['2 Anadh'] = df4['2 Anadh'].replace('#DIV/0!', '0')
df4['1 Anadh'] = df4['1 Anadh'].astype(float)
df4['2 Anadh'] = df4['2 Anadh'].astype(float)

In [6]:
df1

,1 LDF,1 T,1 A365,1 A460,1 Anadh,2 LDF,2 T,2 A365,2 A460,2 Anadh,Sleep stages
0,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
1,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
2,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
3,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
4,19.3,33.72,104.0,65.0,0.625000,20.0,0.0,0.0,0.0,0.0,W
...,...,...,...,...,...,...,...,...,...,...,...
191696,22.0,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191697,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191698,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191699,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2


In [7]:
dataframes = [df1, df2, df3, df4, df5, df6, df7]
label_encoder = LabelEncoder()

for i in range(len(dataframes)):
    dataframes[i] = dataframes[i].dropna(subset=['Sleep stages'])
    dataframes[i] = dataframes[i].fillna(0)
    dataframes[i]['Sleep stages'] = label_encoder.fit_transform(dataframes[i]['Sleep stages'])

df1 = dataframes[0]
df2 = dataframes[1]
df3 = dataframes[2]
df4 = dataframes[3]
df5 = dataframes[4]
df6 = dataframes[5]
df7 = dataframes[6]

In [19]:
df4

,1 LDF,1 T,1 A365,1 A460,1 Anadh,2 LDF,2 T,2 A365,2 A460,2 Anadh,Sleep stages
0,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
1,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
2,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
3,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
4,24.1,36.46,131.0,64.0,0.48855,43.0,36.0,125.0,73.0,0.584,3
...,...,...,...,...,...,...,...,...,...,...,...
228142,0.0,0.00,0.0,0.0,0.00000,44.4,38.8,106.0,58.0,0.000,2
228143,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2
228144,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2
228145,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2


train / val / test

In [9]:
train_data = pd.concat([dataframes[0], dataframes[1], dataframes[3], dataframes[5], dataframes[6]])
val_data = pd.concat([dataframes[4]])
test_data = pd.concat([dataframes[2]])

In [10]:
X_train = train_data.iloc[:, :2].values
y_train = train_data.iloc[:, -1].values

X_val = val_data.iloc[:, :2].values
y_val = val_data.iloc[:, -1].values

X_test = test_data.iloc[:, :2].values
y_test = test_data.iloc[:, -1].values

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

In [ ]:
print(f"Train tensors shape: X_train_tensor: {X_train_tensor.shape}, y_train_tensor: {y_train_tensor.shape}")
print(f"Validation tensors shape: X_val_tensor: {X_val_tensor.shape}, y_val_tensor: {y_val_tensor.shape}")
print(f"Test tensors shape: X_test_tensor: {X_test_tensor.shape}, y_test_tensor: {y_test_tensor.shape}")

Train tensors shape: X_train_tensor: torch.Size([1325931, 2]), y_train_tensor: torch.Size([1325931])
Validation tensors shape: X_val_tensor: torch.Size([369990, 2]), y_val_tensor: torch.Size([369990])
Test tensors shape: X_test_tensor: torch.Size([423721, 2]), y_test_tensor: torch.Size([423721])


полносвязная NN

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Валидация на каждом этапе обучения (каждые 50 эпох)
    if (epoch + 1) % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            _, val_predicted = torch.max(val_outputs.data, 1)

            val_accuracy = accuracy_score(y_val_tensor.numpy(), val_predicted.numpy())
            print(f'Validation Accuracy: {val_accuracy:.4f}')

Epoch [50/1000], Loss: 1.2153
Validation Accuracy: 0.4857
Epoch [100/1000], Loss: 1.2027
Validation Accuracy: 0.4857
Epoch [150/1000], Loss: 1.1897
Validation Accuracy: 0.4857
Epoch [200/1000], Loss: 1.1677
Validation Accuracy: 0.4857
Epoch [250/1000], Loss: 1.1489
Validation Accuracy: 0.4853
Epoch [300/1000], Loss: 1.1438
Validation Accuracy: 0.4834
Epoch [350/1000], Loss: 1.1425
Validation Accuracy: 0.4762
Epoch [400/1000], Loss: 1.1415
Validation Accuracy: 0.4771
Epoch [450/1000], Loss: 1.1406
Validation Accuracy: 0.4751
Epoch [500/1000], Loss: 1.1397
Validation Accuracy: 0.4714
Epoch [550/1000], Loss: 1.1387
Validation Accuracy: 0.4751
Epoch [600/1000], Loss: 1.1381
Validation Accuracy: 0.4770
Epoch [650/1000], Loss: 1.1372
Validation Accuracy: 0.4764
Epoch [700/1000], Loss: 1.1367
Validation Accuracy: 0.4754
Epoch [750/1000], Loss: 1.1362
Validation Accuracy: 0.4770
Epoch [800/1000], Loss: 1.1359
Validation Accuracy: 0.4767
Epoch [850/1000], Loss: 1.1354
Validation Accuracy: 0.477

In [ ]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs.data, 1)

accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
print(f'Accuracy on test set: {accuracy:.4f}')

Accuracy on test set: 0.5301


дерево решений

In [13]:
model2 = DecisionTreeClassifier(random_state=42)
model2.fit(X_train, y_train)

y_val_pred = model2.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.3713


In [15]:
y_test_pred = model2.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.3681


случайный лес

In [22]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

model3 = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=model3, param_grid=param_grid,
                           scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)
best_model3 = grid_search.best_estimator_

y_val_pred = best_model3.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.3649


In [24]:
y_test_pred = best_model3.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.3687
